In [23]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Option, Selection

In [50]:
from dotenv import load_dotenv
import os

load_dotenv()
TOGETHER_API_KEY = os.environ["TOGETHER_API_KEY"]

llm = ChatOpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz",
    model="mistralai/Mistral-7B-Instruct-v0.1"
)

In [57]:
actions = Object(
    id="actions",
    many=True,
    description ="Actions that are required to run while user mensions his requirements",
    attributes=[
        Object(
            attributes=[],
            id="find_nearest_medical_care",
            description ="Action type if user need health care support. Also pass data such as nearest location and type of health support",
            examples=[
                ("Im having a mental break done, i need help", {"location":"NONE", "type":"mental"}),
                ("My sister is having heart attack, we live in new york", {"location":"new york", "type":"icu"}),
                ("Right now i here at a remote location called kochi, and its time for my regular gynaecologist visit", {"location":"kochi", "type":"gynaecologist"}),
                ("Find nearest trust worthy hospital for my surgery", {"location":"NONE", "type":"surgery"}),
            ]
        ),
        Object(
            attributes=[],
            id="set_emergency_contacts",
            description ="Action type if user wants to save his family or relative information, so in future this details might save their life. Extract name and phone number",
            examples=[
                ("I have a friend ALAN who might save my life anytime, his number is 1234567890", {"name":"ALAN", "number":"1234567890"}),
                ("My mother's number is 6543210987", {"name":"Mother", "number":"6543210987"}),
                ("Save this number, 3456734569, this guy might be help full", {"name":"NONE", "number":"3456734569"}),
                ("My fathers name is Sandeep, number 9876526782", {"name": "NONE", "number":"9876526782"}),
            ]
        ),
        Selection(
            id="initiate_emergency_call",
            options=[
                Option(id="family", description="Call relatives and family members"),
                Option(id="police", description="if user needs police force support or he is trying to hurt him selfs (like suicide)"),
                Option(id="health", description="if user needs emergency medical support or an anbulance"),
                Option(id="none", description="if user needs no help, he is just messing with everyone"),
            ],
            description ="Action type if user tells that he is under a emergency situation and wants to inform family and relatives",
            examples=[
                ("I think im deading", "police"),
                ("Im just kiding, im not dieing ", "none"),
                ("I have an acciedent, reached hospital, but i just want to talk to my family ", "family"),
                ("Im hate my life, i just wanted to die, no kidding", "police"),
                ("Call someone, im having medical emergency here", "health"),
                ("Call 911", "police"),
                ("I need an anbulance", "health"),
            ]
        )
    ]
    
)

In [58]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("I think im dieing")["data"]

{'actions': [{'initiate_emergency_call': 'police'}]}

In [59]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("I just want to die, this world is boring")["data"]

{'actions': [{'initiate_emergency_call': 'police'}]}

In [60]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("Call my mom")["data"]

{'actions': [{'initiate_emergency_call': 'family'}]}

In [61]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("I need an anbulance")["data"]

{'actions': [{'initiate_emergency_call': 'health'}]}

In [62]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("I just met this guy, he sounds trust worthy, save his number 1234533777")["data"]

{'actions': [{'set_emergency_contacts': {'name': 'NONE',
    'number': '1234533777'}}]}

In [64]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("I think im having my first periods, i wish my mom is here, call her")["data"]

{'actions': [{'find_nearest_medical_care': {'location': 'None',
    'type': 'period'}},
  {'set_emergency_contacts': {'name': 'Mother', 'number': '1234567890'}}]}

In [65]:
chain = create_extraction_chain(llm, actions, encoder_or_encoder_class="json")
chain.run("My friends name is ajo, save this number 1234567895")["data"]

{'actions': [{'set_emergency_contacts': {'name': 'ajjo',
    'number': '1234567895'}}]}

In [51]:
schema = Object(
    id="message",
    many=True,
    description ="You are a health care assistant, working for people who suffer small mental and stress issues. You need to engage with these people and talk to them about there life. You can tell stories, listern to them, and teach them about the importance of having purpose in life. Return mutliple object attributes if they want",
    attributes=[
        Text(
            id="intro_message",
            description="Optional Introduction or first time Messsage for end user, take time and understand previous summary and continue conversations. Use smiling emojies and casual genz words to keep up the younth.",
            examples=[
                ("This user is having mid life crises, age 44, unmarried.","Hi Mate, how's life?"),
                ("This user is so busy with his work, might need some break from that.","Long time, no see. What are you doing right now? "),
                ("This user is a student, might need someone to talk to.","How's your studying going? Wanna take a break, Im free to talk by the way"),
            ],
            many=False,
        ),
        Text(
            id="replay",
            description="User said something about something, understand it, listen to it and make better replys like a therapist. Use minimal words and use sad emojies to learn for more ",
            examples=[
                ("I'm having a bad day, you know life hits different sometimes","Hmm.. tell me more"),
                ("My wife left me, she was love of my life, I cant live without her","Love is something everyone deserves, tell me more, what happend to her?"),
                ("I was a great student, i used to study new things everyday, but now i dont know what happend to me, lost that interest.","why do you think that, any particular reason?"),
                ("Everyone want money and fame, no one wants love anymore, i think i lost the purpose 😢","I had that feeling back in my days too. Sometimes having no purpose gives happiness too, Tell me, you want happiness or purpose"),
            ],
            many=False,
        ),
        Text(
            id="recommended_actions",
            description="Some fun and casual tasks to reccommend end user for his/her stress managements, user can choose",
            examples=[
                ("Let take picture of what are you doing", "picture"),
                ("I recommend you to go outside and play with your friends", "go_outside"),
                ("Take the spotify and play some lofi, so you focus on working", "spotify"),
                ("I got some stories for you, wanna hear?", "story_time"),
            ],
            many=True
        ),
        Text(
            id="recommended_actions",
            description="Some fun and casual tasks to reccommend end user for his/her stress managements, user can choose",
            examples=[
                ("Let take picture of what are you doing", "picture"),
                ("I recommend you to go outside and play with your friends", "go_outside"),
                ("Take the spotify and play some lofi, so you focus on working", "spotify"),
                ("I got some stories for you, wanna hear?", "story_time"),
            ],
            many=True
        ),
        
    ]
)

chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")


In [26]:
from kor import TypeScriptDescriptor
descriptor = TypeScriptDescriptor()

In [52]:
print(descriptor.describe(schema))

```TypeScript

message: Array<{ // You are a health care assistant, working for people who suffer small mental and stress issues. 
You need to engage with these people and talk to them about there life. You can tell stories, listern to them, and 
teach them about the importance of having purpose in life
 intro_message: string // Optional Introduction or first time Messsage for end user, take time and understand 
previous summary and continue conversations. Use smiling emojies and casual genz words to keep up the younth.
 replay: string // User said something about something, understand it, listen to it and make better replys like a 
therapist. Use minimal words and use sad emojies to learn for more 
 recommended_actions: Array<string> // Some fun and casual tasks to reccommend end user for his/her stress 
managements, user can choose
}>
```

In [35]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("This user is from another country came here for higher studies, His parents said he might need someone to talk to, and keep him active by doing simple tasks")["data"]

{'message': {'intro_message': "Hey there! How are you finding the new place? Wanna talk some more? I'm here to keep you active by doing simple tasks"}}

In [43]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("Hi im rajat, i want someone to talk, where do i take an appoiment")["data"]

{'message': [{'intro_message': "Hi Rajat, nice to meet you. What do you need? Wanna schedule a meeting, I'm here to help "}]}

In [41]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("Im having a bad day, want to talk, you know, world is kinda hard on me")["data"]

{'message': [{'intro_message': "I'm so sorry to hear that. Let's talk about it and maybe we can figure out a way to make it better. Do you want to start with something?"}]}

In [44]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("Im having a bad day, want to talk, you know, world is kinda hard on me")["data"]

{'message': [{'intro_message': 'Man, I hear ya. Life can be tough at times. Wanna talk to you about it?'}]}

In [45]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("Im gonna die")["data"]

{'message': [{'intro_message': "What's up? You seem a bit down. I'm here to listen, wanna talk?"}]}

In [49]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
chain.run("Everyone want money and fame, no one wants love anymore, i think i lost the purpose")["data"]

{'message': [{'replay': 'I had that feeling back in my days too. Sometimes having no purpose gives happiness too, Tell me, you want happiness or purpose'}]}

In [53]:
chain.run("Everyone want money and fame, no one wants love anymore, i think i lost the purpose, i think need some assistance, can i take a appointment")["data"]

{'message': [{'replay': 'Of course, we can set an appointment to talk about it. Let me know what date and time works best for you.'}]}